Rete per la gestione di dei dati con shape [32,seq_len, 768] tramite convoluzione 2D

In [1]:

import numpy as np
from sklearn.model_selection import train_test_split
import json
from torch.utils.data import Dataset
import os
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics



/home/antoniodeblasi/anaconda3/envs/bioInfo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

class MyDataSet(Dataset):
    def __init__(self, path):
        self.path = path
        self.df_sequences = os.listdir(path+'/embeddings')
        self.df_labels = os.listdir(path+'/labels')

    def __len__(self):
        return len(self.df_sequences)

    def __getitem__(self, index):
        sequence = torch.load(self.path+'/embeddings/'+self.df_sequences[index])

        label = torch.load(self.path+'/labels/'+self.df_labels[index]).float()
        

        return sequence, label

In [4]:
DataTrainpath="/home/antoniodeblasi/Scaricati/Dataset_V1"
DataTestpath="/home/antoniodeblasi/Scaricati/Dataset_V1_testing"
DataValpath="/home/antoniodeblasi/Scaricati/Dataset_V1_validation"


train_data_object = MyDataSet(DataTrainpath)
test_data_object = MyDataSet(DataTestpath)
val_data_object = MyDataSet(DataValpath)


def collate(batch):
  (a, b) = batch[0]

  a = a.unsqueeze(1)
  return (a,b)

In [5]:
for seq, _ in torch.utils.data.DataLoader(train_data_object,
        batch_size=1, shuffle = False, collate_fn=collate):
    print(seq.shape)
    break

torch.Size([32, 1, 138, 768])


In [6]:

class MyDataModule(pl.LightningDataModule):

  def setup(self, stage):
    self.dataset = ""#MyDataSet("")


  def train_dataloader(self):
    return torch.utils.data.DataLoader(train_data_object,
        batch_size=1, shuffle = False, collate_fn=collate)
  def val_dataloader(self):
    return torch.utils.data.DataLoader(val_data_object,
        batch_size=1, shuffle = False, collate_fn=collate)
  def test_dataloader(self):
    return torch.utils.data.DataLoader(test_data_object,
       batch_size=1, shuffle = False, collate_fn=collate)


In [9]:


class CNN(pl.LightningModule):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.pool = nn.MaxPool2d(2)
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)  # Pooling globale per ridurre le dimensioni

        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
        
        # for validation/testing
        self.accuracy = torchmetrics.Accuracy(task="binary")

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.relu(self.conv3(x))
        x = self.global_avg_pool(x)
        x = torch.relu(self.fc1(x.squeeze()))
        x = self.sigmoid(self.fc2(x))
        return x

    def cross_entropy_loss(self, logits, labels):
      return F.binary_cross_entropy(logits, labels)

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x).squeeze()
        loss = self.cross_entropy_loss(logits, y)
        self.log('train_loss', loss)
        return loss


    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self.forward(x).squeeze()
        loss = self.cross_entropy_loss(logits, y)
        acc = self.accuracy(logits,y)
        
        self.log('val_loss', loss)
        self.log('val_accuracy', acc)

    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        logits = self.forward(x).squeeze()
        loss = self.cross_entropy_loss(logits, y)
        acc = self.accuracy(logits,y)
        
        self.log('test_loss', loss)
        self.log('test_accuracy', acc)

    def configure_optimizers(self):
      optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.8)
      return optimizer

# Creazione del modello
model = CNN()

In [10]:

trainer = pl.Trainer(max_epochs=40)

data_module = MyDataModule()

trainer.fit(model, data_module)
# Valutazione del modello
p = trainer.test(model, data_module)
print("Loss sul set di validazione:", p)




GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type              | Params
------------------------------------------------------
0 | conv1           | Conv2d            | 320   
1 | conv2           | Conv2d            | 18.5 K
2 | conv3           | Conv2d            | 73.9 K
3 | pool            | MaxPool2d         | 0     
4 | global_avg_pool | AdaptiveAvgPool2d | 0     
5 | fc1             | Linear            | 8.3 K 
6 | fc2             | Linear            | 65    
7 | sigmoid         | Sigmoid           | 0     
8 | accuracy        | BinaryAccuracy    | 0     
------------------------------------------------------
100 K     Trainable params
0         Non-trainable params
100 K     Total params
0.404     Total estimated model params size (MB)


Epoch 39: 100%|██████████| 765/765 [00:57<00:00, 13.38it/s, v_num=43]      

`Trainer.fit` stopped: `max_epochs=40` reached.


Epoch 39: 100%|██████████| 765/765 [00:57<00:00, 13.38it/s, v_num=43]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 96/96 [00:03<00:00, 31.64it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │       0.5322265625        │
│         test_loss         │    0.6908988356590271     │
└───────────────────────────┴───────────────────────────┘

Loss sul set di validazione: [{'test_loss': 0.6908988356590271, 'test_accuracy': 0.5322265625}]
